In [2]:
import time
import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
load_dotenv()


class PbiScrap:
    def __init__(self):
        self.datos_tabla = []
        self.translate_y = 0
        self.incremento = 8.68711
        self.total_filas_anteriores = 0
        self.total_filas_actuales = 0
        self.batch_size = 100

    def make_scroll_and_extract(self, actions, scroll_bar_part, driver):
        try:
            actions.click_and_hold(scroll_bar_part).move_by_offset(0, int(self.incremento)).release().perform()
            time.sleep(2)
            self.extraer_datos_filas(driver)
            return True
        except Exception as e:
            print(f"Error from make_scroll: {e}")
            return False

    def extraer_datos_filas(self, driver):
        try:
            filas = driver.find_elements(By.XPATH, '//div[@role="row"]')
            for index, fila in enumerate(filas):
                if index == 0:
                    continue
                celdas = fila.find_elements(By.XPATH, ".//div[@role='gridcell']")
                datos_fila = [celda.text for celda in celdas]
                self.datos_tabla.append(datos_fila)
        except Exception as e:
            print(f"Error from extraer_datos_filas: {e}")

    def export_data_in_batches(self, data, batch_size):
        try:
            for i in range(0, len(data), batch_size):
                batch = data[i:i + batch_size]
                #batch_count = len(batch)
                #suffix = f"_{i + batch_count}"
                filename = f"silice_data.csv"
                
                df = pd.DataFrame(batch, columns=["Column", "Fecha de Envio", "Campaña", "Télefono", "Nombre Cliente", "hiloId"])
                df.to_csv(r'../data/raw/' + filename, mode='a',index=False, encoding="utf-8")
        except Exception as e:
            print(f"Error en export_data_in_batches: {e}")

    def main(self):
        try : 
            URL = os.getenv('PBI_URL')
            XPATH_TABLE = os.getenv('XPATH_TABLE')
            XPATH_FILTER_DATE = os.getenv('XPATH_FILTER_DATE')

            driver = webdriver.Chrome()
            driver.get(URL)

            wait = WebDriverWait(driver, 30)
            slicer = wait.until(EC.presence_of_element_located((By.XPATH,XPATH_FILTER_DATE)))
                                                                
            filter_init = driver.find_element(By.CSS_SELECTOR, ".noUi-handle-lower")
            filter_end = driver.find_element(By.CSS_SELECTOR, ".noUi-handle-upper") 

            actions = ActionChains(driver)

            desplazamiento_total = -1000
            incremento = -50

            for _ in range(abs(desplazamiento_total) // abs(incremento)):
                try:
                    actions.click_and_hold(filter_init).move_by_offset(incremento, 0).release().perform()
                    time.sleep(1)
                except Exception as e:
                    print(f"Error al mover el slicer: {e}")
                    break

            scroll_bar_div = driver.find_elements(By.CSS_SELECTOR, ".scroll-bar-div")[1]
            scroll_bar_part = scroll_bar_div.find_element(By.CSS_SELECTOR, ".scroll-bar-part-bar")
            actions = ActionChains(driver)

            while self.make_scroll_and_extract(actions, scroll_bar_part, driver):
                if len(self.datos_tabla) >= self.batch_size:
                    self.export_data_in_batches(self.datos_tabla, self.batch_size)
                    self.datos_tabla.clear()

            if self.datos_tabla:
                self.export_data_in_batches(self.datos_tabla, self.batch_size)

        except Exception as e:
            print(f"Error from main: {e}")
        finally:
            driver.quit()


pbi = PbiScrap()
pbi.main()

Error from make_scroll: Message: target frame detached: received Inspector.detached event
  (Session info: chrome=134.0.6998.37)
Stacktrace:
	GetHandleVerifier [0x00007FF7124BFE45+26629]
	(No symbol) [0x00007FF712426010]
	(No symbol) [0x00007FF7122B931A]
	(No symbol) [0x00007FF7122A7DDA]
	(No symbol) [0x00007FF7122A6B92]
	(No symbol) [0x00007FF7122A6656]
	(No symbol) [0x00007FF7122A631A]
	(No symbol) [0x00007FF7122A3E9F]
	(No symbol) [0x00007FF7122A478C]
	(No symbol) [0x00007FF7122A2F86]
	(No symbol) [0x00007FF7122C676E]
	(No symbol) [0x00007FF71236C3F6]
	(No symbol) [0x00007FF712337AAA]
	(No symbol) [0x00007FF712360169]
	(No symbol) [0x00007FF712337883]
	(No symbol) [0x00007FF712300550]
	(No symbol) [0x00007FF712301803]
	GetHandleVerifier [0x00007FF7128172BD+3529853]
	GetHandleVerifier [0x00007FF71282DA22+3621858]
	GetHandleVerifier [0x00007FF7128224D3+3575443]
	GetHandleVerifier [0x00007FF71258B77A+860474]
	(No symbol) [0x00007FF71243088F]
	(No symbol) [0x00007FF71242CBC4]
	(No symbo